# Lab 3: Mastering Prompt Engineering with Gemini 2.0 Flash

**Course:** Introduction to Generative AI and Prompt Engineering

**Duration:** 2 hours

This notebook contains hands-on exercises covering zero-shot, few-shot, role prompting, chain-of-thought, JSON extraction, multi-modal prompting, and self-consistency techniques tailored for `gemini-2.0-flash`.

Each exercise has: 1) prompt template, 2) sample input, 3) expected output (where applicable), and 4) a code cell that students can adapt to call the Gemini API in their environment.

----


## Prerequisites

- Google Colab or local Jupyter with access to Gemini API (or equivalent environment).
- `gemini-2.0-flash` model credentials / API key configured.

If you're using the Google Gemini Python SDK or OpenAI-like wrapper, replace the placeholder call in the example cells with the provider-specific client code.

----


In [1]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv( r"C:\Users\Dr. Bhavesh Dharmani\Documents\PyWork\PGDM_GenAI\myAPI.env"))


True

In [2]:
from openai import OpenAI
client = OpenAI(
    api_key = os.getenv("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# from google import genai
# gemini_client = genai.Client(api_key=os.environ['GEMINI_API_KEY'])

## Phase 1 — Explore (30 min)

### 1. Zero-shot Prompting

**Task:** Summarize a business text in 3 bullet points focusing on sales performance.

**Sample text:**

"Our quarterly sales grew by 15% in North America, driven by online campaigns. However, Asia-Pacific saw a decline due to supply chain disruptions. The new product line 'EcoSmart' contributed 25% of total sales."

### Prompt template (Zero-shot):

```
Summarize the following business report in 3 bullet points focusing on sales performance:

[TEXT HERE]
```


In [6]:
# Example: prepare prompt string

text = (
    "Our quarterly sales grew by 15% in North America, driven by online campaigns. "
    "However, Asia-Pacific saw a decline due to supply chain disruptions. "
    "The new product line 'EcoSmart' contributed 25% of total sales."
)

prompt = f"Summarize the following business report in 3 bullet points focusing on sales performance:\n\n{text}"
# print(prompt)

response = client.chat.completions.create(
    model = "gemini-2.0-flash",    #"gemini-1.5-flash", 
    messages = [{"role": "user", "content": prompt}], 
    max_tokens = 100
)
print(response.choices[0].message.content)

# ---
# Replace the following pseudo-call with actual Gemini API call in your environment.
# Example placeholder (DO NOT run unless you have an API client configured):
# response = gemini_client.complete(model='gemini-2.0-flash', prompt=prompt)
# print(response.text)

# Sample expected output (for instructors):
# expected = "- North America sales increased 15%, driven by online campaigns.\n- Asia-Pacific sales declined due to supply chain disruptions.\n- 'EcoSmart' product line contributed 25% of total sales."
# print('\nExpected output:\n', expected)


Here's a 3-bullet point summary of the sales performance:

*   **Strong Growth in North America:** Quarterly sales increased by 15% in North America, fueled by successful online marketing initiatives.
*   **Asia-Pacific Decline:** Sales in the Asia-Pacific region decreased due to ongoing supply chain challenges.
*   **EcoSmart Success:** The new 'EcoSmart' product line significantly contributed to overall revenue, accounting for 25% of total sales.


### 2. Few-shot Prompting

Provide examples so the model follows the structure for extracting product names and contribution percent.

Example prompt (few-shot):

```
Extract the product name and its contribution from the following texts.

Example 1:
Text: "The AI-powered SmartDesk accounted for 30% of sales."
Output: {"product_name": "SmartDesk", "sales_contribution": "30%"}

Example 2:
Text: "EcoSmart line contributed 25% of total sales."
Output: {"product_name": "EcoSmart", "sales_contribution": "25%"}

Now extract from:
"The UltraCharge battery series achieved 40% of the market share."
```


In [ ]:
# Few-shot prompt creation
examples = '''Extract the product name and its contribution from the following texts.

Example 1:
Text: "The AI-powered SmartDesk accounted for 30% of sales."
Output: {"product_name": "SmartDesk", "sales_contribution": "30%"}

Example 2:
Text: "EcoSmart line contributed 25% of total sales."
Output: {"product_name": "EcoSmart", "sales_contribution": "25%"}

Now extract from:
"The UltraCharge battery series achieved 40% of the market share."'''

print(examples)

# Placeholder for API call
# response = gemini_client.complete(model='gemini-2.0-flash', prompt=examples)
# print(response.text)

# Expected output:
print('\nExpected JSON output:\n', '{"product_name": "UltraCharge", "sales_contribution": "40%"}')


### 3. Role Prompting

Ask the model to adopt a persona to change tone and focus. Compare outputs.

Prompt 1 (Market Research Analyst):
```
You are a market research analyst. Write a concise 100-word insight report based on:
"Our customer feedback shows growing demand for AI-driven automation tools in logistics and warehouse management."
```

Prompt 2 (Journalist): Same input but persona = journalist. Observe differences in tone and priority.


In [ ]:
# Role prompting examples
input_text = "Our customer feedback shows growing demand for AI-driven automation tools in logistics and warehouse management."
prompts = {
    'analyst': f"You are a market research analyst. Write a concise 100-word insight report based on:\n\n{input_text}",
    'journalist': f"You are a journalist writing for a tech business section. Summarize the key insight in ~100 words based on:\n\n{input_text}"
}

for role, p in prompts.items():
    print(f"--- Prompt ({role}) ---\n", p, "\n")

# Replace with real API calls to compare outputs.


## Phase 2 — Learn (45 min)

### 4. Chain-of-Thought Prompting

Ask Gemini to show intermediate steps. Use `show_steps` style instructions or explicitly request reasoning steps.

Prompt:
```
A company sells two products: Alpha and Beta.
Alpha profit = ₹200 per unit, Beta profit = ₹150 per unit.
They sold 180 Alpha and 220 Beta units this month.

Calculate total profit step-by-step and give final result in JSON:
{"Total_Profit": <value_in_rupees>}
```


In [ ]:
# Chain-of-thought example
prompt = (
    "A company sells two products: Alpha and Beta.\n"
    "Alpha profit = ₹200 per unit, Beta profit = ₹150 per unit.\n"
    "They sold 180 Alpha and 220 Beta units this month.\n\n"
    "Calculate total profit step-by-step and give final result in JSON:\n"
    '{"Total_Profit": ""}'
)

print('Prompt prepared (students should send to Gemini):')
print(prompt)

# Expected numeric calculation (computed here for verification):
alpha_profit = 200 * 180
beta_profit = 150 * 220
total_profit = alpha_profit + beta_profit
print('\nExpected JSON output:')
print({'Total_Profit': total_profit})


### 5. JSON Extraction from Business Reports

Input Text (sample invoice):
```
Invoice ID: 9871
Client: BrightTech Pvt. Ltd.
Date: 2025-11-13
Items:
 - AI Server (2 units @ ₹80,000)
 - Edge Sensor Kit (5 units @ ₹12,000)
Payment: Pending
```

Prompt (strict JSON):
```
Extract the following data in strict JSON format:
{
 "Invoice_ID": "",
 "Client_Name": "",
 "Total_Amount": "",
 "Payment_Status": ""
}
```

Students should ensure the model computes totals correctly and returns only valid JSON.


In [ ]:
# Compute total in Python (ground truth) and present prompt
invoice_text = '''Invoice ID: 9871
Client: BrightTech Pvt. Ltd.
Date: 2025-11-13
Items:
 - AI Server (2 units @ ₹80,000)
 - Edge Sensor Kit (5 units @ ₹12,000)
Payment: Pending
'''

# Compute totals programmatically (instructor verification)
ai_server_total = 2 * 80000
edge_sensor_total = 5 * 12000
computed_total = ai_server_total + edge_sensor_total

print('Computed total (INR):', computed_total)

prompt = f"Extract the following data in strict JSON format:\n{{\n 'Invoice_ID': '',\n 'Client_Name': '',\n 'Total_Amount': '',\n 'Payment_Status': ''\n}}\n\nSource:\n{invoice_text}"
print('\nPrepared prompt for Gemini:\n')
print(prompt)

# Expected JSON sample:
expected_json = {
    "Invoice_ID": "9871",
    "Client_Name": "BrightTech Pvt. Ltd.",
    "Total_Amount": f"₹{computed_total}",
    "Payment_Status": "Pending"
}
print('\nExpected JSON output for verification:\n')
print(expected_json)


### 6. Instruction + Few-shot Prompting for Multi-Entity Extraction

Example:
```
Review: "The delivery was quick, but packaging was poor."
Output: {"Delivery": "positive", "Packaging": "negative"}
```

Now try:
```
"Excellent product quality, but after-sales support was disappointing."
```


In [ ]:
# Prepare the few-shot prompt
fewshot = '''Extract customer feedback details from each review and format in JSON.

Example:
Review: "The delivery was quick, but packaging was poor."
Output: {"Delivery": "positive", "Packaging": "negative"}

Now extract from:
"Excellent product quality, but after-sales support was disappointing."'''

print(fewshot)

# Expected output:
print('\nExpected JSON:\n', '{"Product_Quality": "positive", "After_Sales_Support": "negative"}')


## Phase 3 — Consolidate (45 min)

### 7. Multi-modal Prompting (Text + Image)

Provide an image (e.g. sales chart or product lineup) and the accompanying prompt:

```
Analyze the uploaded image and summarize the insights in two sentences.
Then, extract key data points in JSON format:
{
 "Highest_Selling_Product": "",
 "Lowest_Selling_Product": "",
 "Observation": ""
}
```

In Colab, students can use the file upload widget and then pass the image bytes to the Gemini multimodal endpoint.

### 8. Multi-turn Context Retention Test

Turn 1: Provide quarterly sales and ask for summary.
Turn 2: Ask for prediction based on the trend and return JSON only.

### 9. Self-Consistency Prompting

Ask Gemini to reason step-by-step, detect inconsistencies, and return a JSON verdict.


In [ ]:
# Placeholder cells for multimodal / multi-turn examples
# Uploading images in Colab (example):
# from google.colab import files
# uploaded = files.upload()
# image_filename = list(uploaded.keys())[0]
# with open(image_filename, 'rb') as f:
#     image_bytes = f.read()

# Example multimodal prompt (instructors will adapt to their SDK):
multimodal_prompt = (
    "Analyze the uploaded image and summarize the insights in two sentences. "
    "Then, extract key data points in JSON format:\n"
    "{\n 'Highest_Selling_Product': '',\n 'Lowest_Selling_Product': '',\n 'Observation': ''\n}\n"
)
print(multimodal_prompt)

# Multi-turn example (conceptual):
turn1 = 'Q1: 12L, Q2: 15L, Q3: 10L, Q4: 18L. Summarize quarterly trend.'
turn2 = 'Based on that trend, predict next year\'s Q1 assuming a 10% increase from Q4. Return JSON format only: {"Predicted_Q1": value}'
print('\nTurn1:', turn1)
print('Turn2:', turn2)


## Reflection & Deliverables

Each student should submit:

1. `Prompt_Engineering_Lab.ipynb` with outputs for all exercises.
2. A 200-word reflective note: 'What I learned about prompt design.'

Optional advanced challenge: Prompt compression (<40 words) that extracts company name, product line, and total sales.

----


## Instructor notes — Integrating with Gemini API

- Replace placeholder pseudo-calls with your institution's Gemini SDK or API client code.
- When requiring chain-of-thought, some SDKs require `reasoning: 'detailed'` or to append explicit instructions like "Think step-by-step:".
- For strict JSON outputs, add post-processing/validation in Python to ensure returned text is valid JSON (e.g. `json.loads(response)` after small cleaning).
